In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# SVHN Transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load SVHN
train_data = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_data = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# Same 9-layer CNN
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))
        )

        self.fc = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight)
                with torch.no_grad():
                    m.weight *= torch.sqrt(torch.tensor(2.))

    def forward(self, x):
        x = self.layers(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Training function
def train(model, epochs=50):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        correct, total = 0, 0
        model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f} | Test Accuracy: {acc:.2f}%")

# Run it
model = CustomCNN()
train(model)


Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat
Epoch 1 | Loss: 0.7609 | Test Accuracy: 90.17%
Epoch 2 | Loss: 0.2710 | Test Accuracy: 93.09%
Epoch 3 | Loss: 0.2052 | Test Accuracy: 93.49%
Epoch 4 | Loss: 0.1599 | Test Accuracy: 93.76%
Epoch 5 | Loss: 0.1263 | Test Accuracy: 93.35%
Epoch 6 | Loss: 0.0933 | Test Accuracy: 93.61%
Epoch 7 | Loss: 0.0736 | Test Accuracy: 93.36%
Epoch 8 | Loss: 0.0574 | Test Accuracy: 93.82%
Epoch 9 | Loss: 0.0502 | Test Accuracy: 93.95%
Epoch 10 | Loss: 0.0455 | Test Accuracy: 92.90%
Epoch 11 | Loss: 0.0384 | Test Accuracy: 93.10%
Epoch 12 | Loss: 0.0344 | Test Accuracy: 93.57%
Epoch 13 | Loss: 0.0329 | Test Accuracy: 93.15%
Epoch 14 | Loss: 0.0301 | Test Accuracy: 93.52%
Epoch 15 | Loss: 0.0324 | Test Accuracy: 93.68%
Epoch 16 | Loss: 0.0262 | Test Accuracy: 93.53%
Epoch 17 | Loss: 0.0273 | Test Accuracy: 93.62%
Epoch 18 | Loss: 0.0256 | Test Accuracy: 92.55%
Epoch 19 | Loss: 0.0235 | Te

KeyboardInterrupt: 